In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tkwargs = {"dtype": torch.float64, "device": device}
torch.set_default_dtype(torch.float64)
print(f"device {device}")

In [ ]:
from hpob_handler import HPOBHandler
import numpy as np
import matplotlib.pyplot as plt


from dkl_gp import GPModelDKL
from botorch.utils.transforms import normalize, unnormalize, standardize
from gpytorch.mlls import PredictiveLogLikelihood, ExactMarginalLogLikelihood, VariationalELBO
import gpytorch

#### Data from HPO-B library https://github.com/releaunifreiburg/HPO-B

In [ ]:
hpob_hdlr = HPOBHandler(root_dir="hpob-data/", mode="v3")
# search_space_id =  hpob_hdlr.get_search_spaces()[0]
# dataset_id = hpob_hdlr.get_datasets(search_space_id)

# xgboost ids in HPO-B
search_space_ids = ['5906', '5971', '6767']
# 3 random datasets
dataset_ids = ['9957', '145862', '3891']

### Training Data utils 

In [ ]:
def convert_tensor_and_standardize(X, y, bounds=None, Y_mean=None, Y_std=None):
    X = torch.tensor(X, **tkwargs)
    y = torch.tensor(y, **tkwargs).squeeze()
    print(X.shape, y.shape)
    if bounds is None:
        bounds = (torch.stack([torch.min(X, dim=0)[0], torch.max(X, dim=0)[0] + 1e-8]))
    X = normalize(X, bounds=bounds)
    if Y_mean is None:
        stddim = -1 if y.dim() < 2 else -2
        Y_std = y.std(dim=stddim, keepdim=True)
        Y_std = Y_std.where(Y_std >= 1e-9, torch.full_like(Y_std, 1.0))
        Y_mean = y.mean(dim=stddim, keepdim=True)
    y = (y - Y_mean) / Y_std
    print(torch.any(torch.isnan(X)))
    print(torch.any(torch.isnan(y)))
    return X, y, bounds, Y_mean, Y_std

In [ ]:
def get_test_data_by_search_id(search_space_id, dataset_index = 0):
    dataset_keys = list(hpob_hdlr.meta_test_data[search_space_id].keys())
    key = dataset_keys[dataset_index]
    X = hpob_hdlr.meta_test_data[search_space_id][key]["X"]
    y = hpob_hdlr.meta_test_data[search_space_id][key]["y"]
    return X, y

In [ ]:
def get_train_data_by_search_id(search_space_id):
    all_Xs, all_ys = [], []
    dataset_keys = list(hpob_hdlr.meta_train_data[search_space_id].keys())
    for key in dataset_keys:
        all_Xs.extend(hpob_hdlr.meta_train_data[search_space_id][key]["X"])
        all_ys.extend(hpob_hdlr.meta_train_data[search_space_id][key]["y"])
    return all_Xs, all_ys

In [ ]:
def get_validation_data_by_search_id(search_space_id):
    all_Xs, all_ys = [], []
    dataset_keys = list(hpob_hdlr.meta_validation_data[search_space_id].keys())
    for key in dataset_keys:
        all_Xs.extend(hpob_hdlr.meta_validation_data[search_space_id][key]["X"])
        all_ys.extend(hpob_hdlr.meta_validation_data[search_space_id][key]["y"])
    return all_Xs, all_ys

In [ ]:
def get_covering_support_query_sets(X, y, task_idx, support_size=10, query_size=22, seed=0):
    np.random.seed(seed)
    rand_indices = np.random.permutation(len(X))
    X = X[rand_indices]
    y = y[rand_indices]
    batch_size = support_size + query_size
    num_batches = len(X) // batch_size
    data_batches = []
    for i in range(num_batches):
        start_idx = i * batch_size
        support_idx = start_idx + support_size
        end_idx = (i + 1) * batch_size    
        sup_x = X[start_idx:support_idx]
        sup_y = y[start_idx:support_idx]
        que_x = X[support_idx:end_idx]
        que_y = y[support_idx:end_idx]
        assert sup_x.shape[0] == support_size
        assert sup_y.shape[0] == support_size
        assert que_x.shape[0] == query_size
        assert que_y.shape[0] == query_size
        data_batches.append([sup_x, sup_y.unsqueeze(-1), que_x, que_y.unsqueeze(-1), task_idx])
    return data_batches

In [ ]:
def get_episodes_data(
        all_Xs, 
        all_ys, 
        n_episodes=1000,
        task_idx=0, 
        support_size=10, 
        query_size=22
    ):
    """This method collects episode level data.
    """
    all_points = []
    n_episodes_per_sweep = int(len(all_Xs)/(support_size + query_size))
    for uniq_seed in range((n_episodes // n_episodes_per_sweep) + 1):
        batch_data = get_covering_support_query_sets(all_Xs, 
                                                    all_ys, 
                                                    task_idx=task_idx, 
                                                    support_size=support_size, 
                                                    query_size=query_size, 
                                                    seed=uniq_seed)
        # print(len(batch_data))
        all_points.extend(batch_data)
    return all_points[:n_episodes]

### Model training utils

In [ ]:
# def single_update_step(
#     model,
#     mll,
#     optimizer,
#     sup_x,
#     sup_y,
#     que_x,
#     que_y,
# ):
#     """This method trains the model one episode at a time.
#     """
#     model = model.train() 
#     optimizer.zero_grad()
#     output = model(sup_x.cuda(), sup_y.cuda(), que_x.cuda())
#     loss = -mll(output, que_y.squeeze().cuda())
#     loss.backward()
#     # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#     optimizer.step()
#     model = model.eval()

#     return model, loss.item()

In [ ]:
def batch_update_step(
    model,
    mll,
    optimizer,
    batch_data,
):
    """Training loop operating on a batch of episodes.
    """
    model = model.train() 
    optimizer.zero_grad()
    total_loss = 0.0
    for i in range(len(batch_data)):
        sup_x, sup_y, que_x, que_y, task_idx = batch_data[i]
        output = model(sup_x.cuda(), sup_y.cuda(), que_x.cuda())
        loss = -mll(output, que_y.squeeze().cuda())
        total_loss = total_loss + loss
    total_loss = (total_loss / len(batch_data))
    total_loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    model = model.eval()
    return model, total_loss.item()

In [ ]:
def get_validation_loss(model,
                        mll,
                        valid_data):
    model = model.eval() 
    total_loss = 0.0
    for i in range(len(valid_data)):
        sup_x, sup_y, que_x, que_y, task_idx = valid_data[i]
        with torch.no_grad():
            output = model(sup_x.cuda(), sup_y.cuda(), que_x.cuda())
            loss = -mll(output, que_y.squeeze().cuda())
        total_loss = total_loss + loss.item()
    return total_loss / len(valid_data)

In [ ]:
# Acquisition function needs to be defined manually because 
# each pass to the model requires (sup_x, sup_y, que_x)
from botorch.acquisition.analytic import _scaled_improvement, _ei_helper
def ei_acqf(model, inputs, best_f):
    dist = model.posterior(*inputs)
    mean = dist.mean
    sigma = dist.variance ** (1/2)
    u = _scaled_improvement(mean, sigma, best_f, maximize=True)
    return sigma * _ei_helper(u)

In [ ]:
# training data setup
all_train_Xs, all_train_ys = get_train_data_by_search_id(search_space_ids[0])
print(len(all_train_Xs), len(all_train_ys))
(
    all_train_Xs, 
    all_train_ys, 
    train_bounds, 
    train_Y_mean, 
    train_Y_std
)   = convert_tensor_and_standardize(all_train_Xs, all_train_ys)
episodes_data = get_episodes_data(all_train_Xs, all_train_ys, n_episodes=2000)
print(len(episodes_data))

In [ ]:
# validation data setup
all_valid_Xs, all_valid_ys = get_validation_data_by_search_id(search_space_ids[0])
all_valid_Xs, all_valid_ys, _, _, _ = convert_tensor_and_standardize(all_valid_Xs, 
                                                            all_valid_ys, 
                                                            bounds=train_bounds,
                                                            Y_mean=train_Y_mean,
                                                            Y_std=train_Y_std
                                                            )
validation_episodes_data = get_covering_support_query_sets(
                                                    all_valid_Xs, 
                                                    all_valid_ys, 
                                                    task_idx=0, 
                                                    support_size=22, 
                                                    query_size=10, 
                                                    seed=0
                            )
# sup_x, sup_y, que_x, que_y, task_idx = episodes_data[0]
# for episode in validation_episodes_data:
#     episode[0] = sup_x
#     episode[1] = sup_y
print(len(validation_episodes_data))

In [ ]:
# test data setup
all_test_Xs, all_test_ys = get_test_data_by_search_id(search_space_ids[0])
all_test_Xs, all_test_ys, _, _, _ = convert_tensor_and_standardize(all_test_Xs, all_test_ys)
test_episodes_data = get_covering_support_query_sets(all_test_Xs, 
                                                    all_test_ys, 
                                                    task_idx=1, 
                                                    support_size=10, 
                                                    query_size=10, 
                                                    seed=0)
print(len(test_episodes_data))

In [ ]:
# episodes_data.append(test_episodes_data[0])
# episodes_data.append(validation_episodes_data[0])

### Hyperparameters to choose for the model
- Episode parameters:
    - support_size: 10
    - query_size: 22
    - n_episodes: 1000
- Batch size (at the episode level): 32

In [ ]:
ind_pts_size = 20
np.random.seed(0)
rand_indices = np.random.permutation(len(episodes_data))[:ind_pts_size]
inducing_points = [episodes_data[idx] for idx in rand_indices]
print(len(inducing_points))

In [ ]:
likelihood =  gpytorch.likelihoods.GaussianLikelihood().cuda() 
gp_model = GPModelDKL(inducing_points_set=inducing_points, 
                      likelihood=likelihood, 
                      network_dims=(64, 64),
                      learn_inducing_locations=True)

In [ ]:
learning_rte = 0.001
optimizer = torch.optim.Adam([{'params': gp_model.parameters(), 'lr': learning_rte} ], lr=learning_rte)
mll = PredictiveLogLikelihood(gp_model.likelihood, gp_model, num_data=len(episodes_data) ** 22).cuda()

In [ ]:
all_losses = []
validation_losses = []
batch_size = 32
for n_epochs in range(500):
    for i in range(0, len(episodes_data) // batch_size, batch_size):
        gp_model, loss = batch_update_step(gp_model, 
                                           mll, 
                                           optimizer, 
                                           episodes_data[i * batch_size: (i+1)*batch_size] 
                        )
        validation_losses.append(get_validation_loss(gp_model, mll, validation_episodes_data))
        all_losses.append(loss)
        if len(all_losses) % 50 == 0 and len(all_losses) != 0:
            # all_losses = np.array(all_losses)
            plt.plot(all_losses[10:], label='training loss')
            plt.legend()
            plt.show()
            # plt.plot(validation_losses, label='validation loss')
            # plt.legend()
            # plt.show()